In [ ]:
from shared.preface import *
import shared.functions as fct

## Ch. 0: Preliminary Investigations.

### 2D grid, center coord. pairs.

In [ ]:
def grid_2D(l, s):

    # 2D spatial grid, discretized.
    eps = s/10
    x, y = np.mgrid[-l:l+eps:s, -l:l+eps:s]

    # [x y] edge coordinate pairs of above grid.
    xy = np.mgrid[-l:l+eps:s, -l:l+eps:s].reshape(2,-1).T

    # Create center coord.-pairs.
    x_centers = (x[1:,:] + x[:-1,:])/2.
    y_centers = (y[:,1:] + y[:,:-1])/2.
    centers = np.array([x_centers[:,:-1], y_centers[:-1,:]])
    cent_coordPairs2D = centers.reshape(2,-1).T
    print('All coord. pairs 2D:\n', cent_coordPairs2D)
    # print('Coord pairs 2D shape:', cent_coordPairs2D.shape)

    return cent_coordPairs2D

limit_coarse, space_coarse = 1.5, 1.
cent_coordPairs2D = grid_2D(limit_coarse, space_coarse)

# Delete middle square.
cent_coordPairs2D = np.delete(cent_coordPairs2D, 4, axis=0)

# Create finegrained square.
limit_fine, space_fine = limit_coarse/2., space_coarse/2.
cent_coordPairs2D_fine = grid_2D(limit_fine, space_fine)

# Insert finegrained square.

### 3D grid, center coord. pairs.

In [ ]:
# 3D spatial grid, discretized.
x, y, z = np.mgrid[-1:1.1:1., -1:1.1:1., -1:1.1:1.]
# print(x[0,...], x.shape)

x_centers = (x[1:,...] + x[:-1,...])/2.
# print(x_centers, x_centers.shape)

y_centers = (y[:,1:,:] + y[:,:-1,:])/2.
# print(y_centers, y_centers.shape)

z_centers = (z[...,1:] + z[...,:-1])/2.
# print(z_centers, z_centers.shape)


# Create center coord.-pairs., truncate redundant points.
centers3D = np.array([
    x_centers[:,:-1,:-1], 
    y_centers[:-1,:,:-1], 
    z_centers[:-1,:-1,:]
])
# print(centers3D, centers3D.shape)

cent_coordPairs3D = centers3D.reshape(3,-1).T 
print(cent_coordPairs3D, cent_coordPairs3D.shape)

## Ch. 1: Milky Way-type halo and simple grid.

### 3D plot of the DM particles.

In [ ]:
snap = '0036'
sim_ID = 'L006N188'
m0 = '2.59e+11'
proj = 2

# Uncomment for interactive 3D plot.
# %matplotlib widget

# Generate files with positions of DM particles
fct.read_DM_positions(
    which_halos='halos', mass_select=12,  # unnecessary when giving index...
    random=False, snap_num=snap, sim=sim_ID, 
    halo_index=int(proj), init_m=m0
)

# Build grid around Milky Way.
MW_grid = fct.grid_3D(GRID_L, GRID_S) / kpc

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

# Read in DM particle positions.
DM_pos = np.load(
    f'CubeSpace/DM_positions_{sim_ID}_snapshot_{snap}_{m0}Msun.npy'
)


print(f'DM particles in halo: {len(DM_pos)}')
print('DM max X coord:', np.max(np.abs(DM_pos[:,0])))
print('DM max Y coord:', np.max(np.abs(DM_pos[:,1])))
print('DM max Z coord:', np.max(np.abs(DM_pos[:,2])))


x_DM, y_DM, z_DM = DM_pos[:,0], DM_pos[:,1], DM_pos[:,2]
cut = 100
x, y, z = x_DM[1::cut], y_DM[1::cut], z_DM[1::cut]

ax.scatter(x, y, z, alpha=0.1, s=5, c='dodgerblue')

# Draw sphere around GC with radius=Rvir_MW.
rGC = Rvir_MW/kpc
uGC, vGC = np.mgrid[0:2 * np.pi:200j, 0:np.pi:100j]
xGC = rGC * np.cos(uGC) * np.sin(vGC)
yGC = rGC * np.sin(uGC) * np.sin(vGC)
zGC = rGC * np.cos(vGC)

xg, yg, zg = MW_grid[:,0], MW_grid[:,1], MW_grid[:,2] 
ax.scatter(xg, yg, zg, s=0.2, marker='x', color='black', alpha=0.5)

print('Cell max X coord:', np.max(np.abs(xg)))
print('Cell max Y coord:', np.max(np.abs(yg)))
print('Cell max Z coord:', np.max(np.abs(zg)))

ax.plot_surface(
    xGC, yGC, zGC, alpha=0.1, 
    cmap=plt.cm.coolwarm, vmin=-1, vmax=1,# antialiased=False,
    rstride=1, cstride=1
)

plt.show()

### Calculate gravity in each cell.

In [ ]:
def cell_gravity(cell_coords, DM_coords, grav_range, m_DM):
    
    # Center all DM positions w.r.t. cell center.
    DM_cc = DM_coords*kpc - cell_coords

    # Calculate distances of DM to cc, sorted in ascending order.
    DM_dist = np.sqrt(np.sum(DM_cc**2, axis=1))

    # Ascending order indices.
    ind = DM_dist.argsort()

    # Truncate DM positions depending on distance to cc.
    DM_pos_sort = DM_cc[ind]
    DM_dist_sort = DM_dist[ind]

    #note: there is a particle exactly at (0,0,0), which causes infinities...
    #? for now manually remove it and discuss with Camila...
    DM_pos_sort = DM_pos_sort[1:]
    DM_dist_sort = DM_dist_sort[1:]

    if grav_range is None:
        DM_pos_inRange = DM_pos_sort
        DM_dist_inRange = DM_dist_sort
    else:
        DM_pos_inRange = DM_pos_sort[DM_dist_sort <= grav_range]
        DM_dist_inRange = DM_dist_sort[DM_dist_sort <= grav_range]

    # Adjust the distances array to make it compatible with DM positions array.
    DM_dist_inRange_sync = DM_dist_inRange.reshape(len(DM_dist_inRange),1)
    DM_dist_inRange_rep = np.repeat(DM_dist_inRange_sync, 3, axis=1)

    ### Calculate superposition gravity.
    pre = G*m_DM
    quotient = (cell_coords-DM_pos_inRange)/(DM_dist_inRange_rep**3)
    derivative = pre*np.sum(quotient, axis=0)

    #NOTE: Minus sign, s.t. velocity changes correctly (see GoodNotes).
    return np.asarray(-derivative, dtype=np.float64)


DM_pos = np.load('sim_data/DM_positions_halos_M12.npy')

# For position of Sun/Earth.
cell_coords = np.array([8.5, 0, 0])*kpc
cell_vector = cell_gravity(cell_coords, DM_pos, GRAV_RANGE, DM_SIM_MASS)
cell_vector /= (kpc/s**2)  
print('Position of Sun/Earth:')
print(cell_vector)
print(np.sqrt(np.sum(cell_vector**2)), '\n')

# For position of Sun/Earth.
cell_coords = np.array([0., 0, 0])*kpc
cell_vector = cell_gravity(cell_coords, DM_pos, GRAV_RANGE, DM_SIM_MASS)
cell_vector /= (kpc/s**2)  
print('Position close to center:')
print(cell_vector)
print(np.sqrt(np.sum(cell_vector**2)), '\n')

# For position of Sun/Earth.
cell_coords = np.array([333., 0, 0])*kpc
cell_vector = cell_gravity(cell_coords, DM_pos, GRAV_RANGE, DM_SIM_MASS)
cell_vector /= (kpc/s**2)  
print('Position at virial radius:')
print(cell_vector)
print(np.sqrt(np.sum(cell_vector**2)))

#! values should be around same order of magnitude as in spher. symmetric setup.

## Ch. 2: Broadcasting for cell_gravity: tests for gravity_range.

### Comparing finite to infinite grav_range:

In [ ]:
snap = '0036'
sim_ID = 'L006N188'
m0s = ['1.89e+12', '4.32e+11', '2.59e+11']
projs = [0,1,2]

for m0, proj in zip(m0s, projs):

    # Comment out to iterate over all halos in m0s.
    if proj != 2:
        continue

    # Generate files with positions of DM particles
    fct.read_DM_positions(
        which_halos='halos', mass_select=12,  # unnecessary when giving index...
        random=False, snap_num=snap, sim=sim_ID, 
        halo_index=int(proj), init_m=m0
    )

    # Read in DM particle positions.
    DM_raw = np.load(
        f'CubeSpace/DM_positions_{sim_ID}_snapshot_{snap}_{m0}Msun.npy'
    )[::1]*kpc
    # print(len(DM))


adapted_cc = np.load(
    f'CubeSpace/adapted_cc_{SIM_ID}_snapshot_{snap}.npy'
)
cell_com = np.load(
    f'CubeSpace/cell_com_{SIM_ID}_snapshot_{snap}.npy'
)
DM_count = np.load(
    f'CubeSpace/DM_count_{SIM_ID}_snapshot_{snap}.npy'
)
DM_pos = np.expand_dims(DM_raw, axis=0)
adapted_DM = np.repeat(DM_pos, len(adapted_cc), axis=0)


fig, ax = plt.subplots(1,1, figsize=(8,8))


ranges = [GRID_S/2., GRID_S/np.sqrt(2)]
labels = ['Inscribed sphere', 'Circumscribed sphere']
for rangeX, l0 in zip(ranges, labels):

    fct.cell_gravity_3D(
        adapted_cc, cell_com, adapted_DM, DM_count, 
        None, DM_SIM_MASS, snap
    )
    dPsi_grid_None = np.load(f'CubeSpace/dPsi_grid_snapshot_{snap}.npy')
    mags_None = np.sqrt(np.sum(dPsi_grid_None**2, axis=1))

    fct.cell_gravity_3D(
        adapted_cc, cell_com, adapted_DM, DM_count, 
        rangeX, DM_SIM_MASS, snap
    )
    dPsi_grid_rangeX = np.load(f'CubeSpace/dPsi_grid_snapshot_{snap}.npy')
    mags_rangeX = np.sqrt(np.sum(dPsi_grid_rangeX**2, axis=1))

    # Sort cells by distance from center (0,0,0).
    grid_dis = np.sqrt(np.sum(adapted_cc**2, axis=2)).flatten()
    dis_ind = grid_dis.argsort()
    grid_dis = grid_dis[dis_ind]
    mags_None = mags_None[dis_ind]
    mags_rangeX = mags_rangeX[dis_ind]

    diff = (mags_None-mags_rangeX)/mags_None
    ax.scatter(
        grid_dis/kpc, diff, s=5, alpha=0.8, 
        label=f'grav_range {np.round(rangeX/kpc,1)} kpc ({l0})'
        )

ax.set_title(
    f'Difference of limited vs. unlimited grav_range (1 good, 0 bad)'
    '\n'
    'Multiple dots for each x-axis point, since multiple cells share same distance'
    )
ax.set_xlabel(f'Cell distance from center (kpc)')
ax.set_ylabel(f'grav. strength w.r.t "infinite" (all DM particles) range')
# ax.set_ylim(0,1)

def y_fmt_here(value, tick_number):
    return np.round(1-value,1)

ax.yaxis.set_major_formatter(ticker.FuncFormatter(y_fmt_here))


plt.legend(loc='lower right')
plt.show()

### Check sim_vs_NFW_gravity.py for comparison to NFW.

## Ch. 3: Better derivative grid structure. 

In [ ]:
# Values for file reading.
sim_ID = 'L006N188'
snap = '0036'
m0 = '2.59e+11'

# Initial grid and DM positions. 
DM_raw = np.load(
    f'CubeSpace/DM_positions_{sim_ID}_snapshot_{snap}_{m0}Msun.npy'
)*kpc  #! needs to be in kpc
grid = fct.grid_3D(GRID_L, GRID_S)
init_cc = np.expand_dims(grid, axis=1)
DM_pos = np.expand_dims(DM_raw, axis=0)
DM_pos_for_cell_division = np.repeat(DM_pos, len(init_cc), axis=0)

DM_lim = 1000

cell_division_count = fct.cell_division(
    init_cc, DM_pos_for_cell_division, GRID_S, DM_lim, None,
    sim=sim_ID, snap_num=snap
    )
adapted_cc = np.load(f'CubeSpace/adapted_cc_{sim_ID}_snapshot_{snap}.npy')

print(init_cc.shape)
print(adapted_cc.shape)

print(f'Total cell division rounds: {cell_division_count}')

### Plotting the outcome after one iteration.

In [ ]:
# Build grid around Milky Way.
new_grid = np.squeeze(adapted_cc, axis=1) / kpc

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

# Read in DM particle positions.
DM_pos = np.load(
    f'CubeSpace/DM_positions_{sim_ID}_snapshot_{snap}_{m0}Msun.npy'
)
print(f'{len(DM_pos)}')
x_DM, y_DM, z_DM = DM_pos[:,0], DM_pos[:,1], DM_pos[:,2]
cut = 10
x, y, z = x_DM[1::cut], y_DM[1::cut], z_DM[1::cut]

ax.scatter(x, y, z, alpha=0.1, c='dodgerblue')

# Draw sphere around GC with radius=Rvir_MW.
rGC = Rvir_MW/kpc
uGC, vGC = np.mgrid[0:2 * np.pi:200j, 0:np.pi:100j]
xGC = rGC * np.cos(uGC) * np.sin(vGC)
yGC = rGC * np.sin(uGC) * np.sin(vGC)
zGC = rGC * np.cos(vGC)

xg, yg, zg = new_grid[:,0], new_grid[:,1], new_grid[:,2] 
ax.scatter(xg, yg, zg, s=0.2, marker='x', color='black', alpha=0.5)


ax.plot_surface(
    xGC, yGC, zGC, alpha=0.1, 
    cmap=plt.cm.coolwarm, vmin=-1, vmax=1,# antialiased=False,
    rstride=1, cstride=1
)

# How many DM particles inside Virial Radius?
DM_dists = np.sqrt(np.sum(DM_pos**2, axis=1))
inside_VR = DM_pos[DM_dists <= rGC]
print(len(inside_VR))

plt.show()

## Ch. 4: Tracing halo masses through snapshots.

! Now a seperate file: merger_tree.py, to generate the MergerTree file.

In [ ]:
sim_ID = 'L006N188'

# Path to merger_tree file.
tree_path = f'{pathlib.Path.cwd().parent}/neutrino_clustering_output_local/MergerTree/MergerTree_{sim_ID}.hdf5'

with h5py.File(tree_path) as tree:
    choice = 1  #note: 0 is ~1e12Msun, 1 & 2 are ~1e11Msun
    Masses = tree['Assembly_history/Mass'][choice,:]
    zeds = tree['Assembly_history/Redshift']
    
    y = np.asarray(Masses)
    x = np.asarray(zeds)

    fig = plt.figure()
    plt.semilogy(x, y, label=f'{y[0]:.2e}')
    plt.title(u'Starting mass in labels ->')
    plt.xlabel('redshift')
    plt.ylabel('halo masses [Msun]')
    plt.legend()
    plt.show()

## Ch. 5: Long-range gravity.

In [7]:
from shared.preface import *
import shared.functions as fct


def check_grid(init_cc, DM_pos, parent_GRID_S, DM_lim):
    """
    Determine which cells have DM above threshold and thus need division.
    """

    # Center all DM positions w.r.t. center, for all cells.
    DM_pos -= init_cc

    # Cell length of current grid generation, used to limit DM particles.
    cell_len = np.ones((len(init_cc),1), dtype=np.float64)*parent_GRID_S/2.

    # Select DM particles inside each cell based on cube length generation.
    DM_in_cell_IDs = np.asarray(
        (np.abs(DM_pos[:,:,0]) <= cell_len) & 
        (np.abs(DM_pos[:,:,1]) <= cell_len) & 
        (np.abs(DM_pos[:,:,2]) <= cell_len)
    )

    # Set DM outside cell to nan values.
    DM_pos[~DM_in_cell_IDs] = np.nan

    DM_sort = np.sort(DM_pos, axis=1)

    # Calculate distances of DM and adjust array dimensionally.
    DM_dis = np.expand_dims(np.sqrt(np.sum(DM_sort**2, axis=2)), axis=2)

    # Drop "rows" common to all cells, which contain only nan values. This is 
    # determined by the cell with the most non-nan entries.
    max_DM_rank = np.max(np.count_nonzero(~np.isnan(DM_sort[:,:,0]), axis=1))
    DM_compact = np.delete(DM_sort, np.s_[max_DM_rank+1:], axis=1)
    del DM_sort

    # Count the number of DM particles in each cell, before deleting cells.
    DM_count_raw = np.count_nonzero(~np.isnan(DM_compact[:,:,0]), axis=1)

    # Calculate c.o.m coords. for each cell, before deleting cells.
    DM_count = np.expand_dims(DM_count_raw, axis=1)
    del DM_count_raw
    DM_count[DM_count==0] = 1  # to avoid divide by zero
    cell_com = np.nansum(DM_compact, axis=1)/DM_count
    del DM_count

    # Count again, overwriting zeros to ones causes issues
    DM_count_final = np.count_nonzero(~np.isnan(DM_compact[:,:,0]), axis=1)

    # Drop all cells containing an amount of DM below the given threshold, 
    # from the DM positions array.
    cell_cut_IDs = DM_count_final <= DM_lim
    DM_cc_minimal = np.delete(DM_compact, cell_cut_IDs, axis=0)
    del DM_compact
    thresh = np.size(DM_cc_minimal, axis=0)

    return DM_count_final, cell_com, cell_cut_IDs, DM_cc_minimal, thresh


def cell_division(
    init_cc, DM_pos, parent_GRID_S, DM_lim, stable_cc, sim, snap_num
    ):

    # Initiate counters.
    thresh = 1
    cell_division_count = 0

    DM_count_arr = []
    cell_com_arr = []
    cell_gen_arr = []

    while thresh > 0:

        DM_count, cell_com, cell_cut_IDs, DM_cc_minimal, thresh = check_grid(
            init_cc, DM_pos, parent_GRID_S, DM_lim
        )

        #! If no cells are in need of division -> return final coords.
        if thresh == 0:

            # Append cell generation number of last iteration.
            cell_gen_arr.append(
                np.zeros(len(init_cc), int) + cell_division_count
            )
            
            # Convert nested lists to ndarray.
            cell_gen_np = np.array(
                list(itertools.chain.from_iterable(cell_gen_arr))
            )

            if cell_division_count > 0:

                # Append DM count and c.o.m. coords of last iteration.
                DM_count_arr.append(DM_count)
                cell_com_arr.append(cell_com)

                # Convert nested lists to ndarray.
                DM_count_np = np.array(
                    list(itertools.chain.from_iterable(DM_count_arr))
                )
                cell_com_np = np.array(
                    list(itertools.chain.from_iterable(cell_com_arr))
                )

                # The final iteration is a concatenation of the survival cells 
                # from the previous iteration and the newest sub8 cell coords 
                # (which are now init_cc).
                final_cc = np.concatenate((stable_cc, init_cc), axis=0)
                np.save(
                    f'CubeSpace/adapted_cc_{sim}_snapshot_{snap_num}.npy', 
                    final_cc
                )
                np.save(
                    f'CubeSpace/DM_count_{sim}_snapshot_{snap_num}.npy',
                    DM_count_np
                )
                np.save(
                    f'CubeSpace/cell_com_{sim}_snapshot_{snap_num}.npy',
                    cell_com_np
                )
                np.save(
                    f'CubeSpace/cell_gen_{sim}_snapshot_{snap_num}.npy',
                    cell_gen_np
                )
                return cell_division_count
            else:

                # Return initial grid itself, if it's fine-grained already.
                np.save(
                    f'CubeSpace/adapted_cc_{sim}_snapshot_{snap_num}.npy', 
                    init_cc
                )
                np.save(
                    f'CubeSpace/DM_count_{sim}_snapshot_{snap_num}.npy',
                    DM_count
                )
                np.save(
                    f'CubeSpace/cell_com_{sim}_snapshot_{snap_num}.npy',
                    cell_com
                )
                np.save(
                    f'CubeSpace/cell_gen_{sim}_snapshot_{snap_num}.npy',
                    cell_gen_np
                )
                return cell_division_count

        else:

            # Save DM count and c.o.m coords of each cell, for stable grid.
            DM_count_arr.append(np.delete(DM_count, ~cell_cut_IDs, axis=0))
            cell_com_arr.append(np.delete(cell_com, ~cell_cut_IDs, axis=0))
            del DM_count, cell_com

            # Array containing all cells (i.e. their coords. ), which need to
            # be divided into 8 "child cells", hence the name "parent cells".
            parent_cc = np.delete(init_cc, cell_cut_IDs, axis=0)

            # "Reset" the DM coords, s.t. all DM positions are w.r.t. the 
            # origin of (0,0,0) again. This way we can easily center them on 
            # the new child cells again, as done in later steps below.
            DM_cc_reset = DM_cc_minimal + parent_cc


            # -------------------------------------------------- #
            # Replace each parent cell by the 8 new child cells. #
            # -------------------------------------------------- #

            # Repeat each DM "column" 8 times. This will be the DM position 
            # array in the next iteration.
            DM_raw8 = np.repeat(DM_cc_reset, repeats=8, axis=0)

            # Create 8 new cells around origin of (0,0,0). The length and size 
            # of the new cells is determined by the previous length of the 
            # parent cell.
            sub8_GRID_S = parent_GRID_S/2.
            sub8_raw = fct.grid_3D(sub8_GRID_S, sub8_GRID_S)

            # Match dimensions of child-array(s) to parent-array(s).
            sub8 = np.expand_dims(sub8_raw, axis=0)
            sub8 = np.repeat(sub8, len(parent_cc), axis=0)

            # Center child-array(s) on parent cell coords.
            sub8_coords = sub8 - parent_cc

            # Reshape array to match repeated DM position array.
            sub8_coords = np.reshape(sub8_coords, (len(parent_cc)*8, 3))
            sub8_coords = np.expand_dims(sub8_coords, axis=1)

            # Delete all cells in initial cell coords array, corresponding to 
            # the cells in need of division, i.e. the parent cells.
            no_parents_cc = np.delete(init_cc, ~cell_cut_IDs, axis=0)

            # Save generation index of stable cells.
            cell_gen_arr.append(
                np.zeros(len(no_parents_cc), int) + cell_division_count
            )

            if cell_division_count > 0:
                stable_cc_so_far = np.concatenate((stable_cc, no_parents_cc), axis=0)
            else:  # ending of first division loop
                stable_cc_so_far = no_parents_cc

            # Overwrite variables for next loop.
            init_cc       = sub8_coords
            DM_pos        = DM_raw8
            parent_GRID_S = sub8_GRID_S
            stable_cc     = stable_cc_so_far

            cell_division_count += 1

# Parameters.
snap = '0036'
m0 = '2.59e+11'
f = 'CubeSpace'
DM_LIM = 50000

# Input data.
grid = fct.grid_3D(GRID_L, GRID_S)
init_cc = np.expand_dims(grid, axis=1)
DM_raw = np.load(f'{f}/DM_positions_{SIM_ID}_snapshot_{snap}_{m0}Msun.npy')*kpc
DM_pos = np.expand_dims(DM_raw, axis=0)
DM_ready = np.repeat(DM_pos, len(init_cc), axis=0)
print('Input data shapes', init_cc.shape, DM_ready.shape)


cell_division_count = cell_division(
    init_cc, DM_ready, GRID_S, DM_LIM, 
    stable_cc=None, sim=SIM_ID, snap_num=snap
)
print(f'cell division rounds: {cell_division_count}')

# Output.
adapted_cc = np.load(
    f'CubeSpace/adapted_cc_{SIM_ID}_snapshot_{snap}.npy')
cell_gen = np.load(
    f'CubeSpace/cell_gen_{SIM_ID}_snapshot_{snap}.npy')
cell_com = np.load(
    f'CubeSpace/cell_com_{SIM_ID}_snapshot_{snap}.npy')
DM_count = np.load(
    f'CubeSpace/DM_count_{SIM_ID}_snapshot_{snap}.npy')

Input data shapes (343, 1, 3) (343, 150358, 3)
cell division rounds: 1


In [8]:
print(DM_count.shape)
print(DM_count.sum())

#! the sum of DM_count should always equal the nr. of DM particles in the halo,
#! no matter how many cell division rounds took place...

(350,)
140849


In [1]:
from shared.preface import *
import shared.functions as fct

def DM_in_cell(cell_coords, cell_gen, DM_pos, grid_s):

        # Center all DM positions w.r.t. cell center.
        DM_pos -= cell_coords

        # Cell lengths to limit DM particles. Limit for the largest cell is 
        # grid_s/2, not just grid_s, therefore the cell_gen+1 !
        cell_len = np.expand_dims(grid_s/(2**(cell_gen+1)), axis=1)

        # Select DM particles inside each cell based on cube length generation.
        DM_in_cell = np.asarray(
            (np.abs(DM_pos[:,:,0]) <= cell_len) & 
            (np.abs(DM_pos[:,:,1]) <= cell_len) & 
            (np.abs(DM_pos[:,:,2]) <= cell_len)
        )

        DM_pos[~DM_in_cell] = np.nan

        return DM_pos


# Parameters.
snap = '0036'
m0 = '2.59e+11'

# Input data.
f = 'CubeSpace'
adapted_cc = np.load(f'{f}/adapted_cc_{SIM_ID}_snapshot_{snap}.npy')
cell_gen = np.load(f'{f}/cell_gen_{SIM_ID}_snapshot_{snap}.npy')
cell_com = np.load(f'{f}/cell_com_{SIM_ID}_snapshot_{snap}.npy')
DM_raw = np.load(f'{f}/DM_positions_{SIM_ID}_snapshot_{snap}_{m0}Msun.npy')*kpc
DM_temp = np.expand_dims(DM_raw, axis=0)
adapted_DM = np.repeat(DM_temp, len(adapted_cc), axis=0)
print('Input data shapes', adapted_cc.shape, cell_gen.shape, cell_com.shape, adapted_DM.shape)

DM_in_cell = DM_in_cell(adapted_cc, cell_gen, adapted_DM, GRID_S)
print('Function output:', DM_in_cell.shape)

non_nans = np.count_nonzero(~np.isnan(DM_in_cell[:,:,0]))
print(non_nans)

Input data shapes (574, 1, 3) (574,) (574, 3) (574, 150358, 3)
Function output: (574, 150358, 3)
140526


In [ ]:
# Parameters.
snap = '0036'
m0 = '2.59e+11'

adapted_cc = np.load(f'CubeSpace/adapted_cc_{SIM_ID}_snapshot_{snap}.npy')
DM_raw = np.load(
            f'CubeSpace/DM_positions_{SIM_ID}_snapshot_{snap}_{m0}Msun.npy'
        )*kpc

# np.float64(DM_raw[0,0])
# DM_raw[0,1]/kpc*1000000000000

DM_temp = np.expand_dims(DM_raw, axis=0)
adapted_DM = np.repeat(DM_temp, len(adapted_cc), axis=0)
adapted_DM -= adapted_cc

print(adapted_cc[0,...] == adapted_cc[0,...]+(0.0001*kpc))
# adapted_DM.shape
# adapted_DM[:,:,0][:2,:5]/kpc
# xyz = 2
# np.allclose(adapted_DM[:,:,xyz][0,:], adapted_DM[:,:,xyz][1,:])
# np.unique(adapted_DM[:,:,xyz][0,:] == adapted_DM[:,:,xyz][-1,:])

#! It seems there is no need to worry about floating point errors!
#! The adapted_DM -= adapted_cc operation returns large enough differences.

In [ ]:
x = np.asarray(np.arange(12).reshape(3,2,2), dtype=np.float64)
x[:2,0,:] = np.nan
print(x)

np.max(np.count_nonzero(~np.isnan(x[:,:,0]), axis=1))
# np.max(np.count_nonzero(~np.isnan(DM_sort[:,:,0])).flatten())

## Ch. 6: More DM structures.